In [74]:
par_storage = "testsynpriv2stor" # default storage account attached to Synapse, private link connection between Synapse and storage account created
par_container = "test"
par_csv_file = 'testcsv.txt'

StatementMeta(test, 2, 86, Finished, Available)

In [75]:
import socket

addr = socket.gethostbyname(par_storage + '.dfs.core.windows.net')

print(addr)

StatementMeta(test, 2, 87, Finished, Available)

10.250.0.10

In [76]:
%%pyspark
# Proof that synapse workspace can load data from attached storage account using private IP
df = spark.read.load('abfss://' + par_container + '@' + par_storage + '.dfs.core.windows.net/' + par_csv_file, format='text')
df.limit(10).show()

StatementMeta(test, 2, 88, Finished, Available)

+-------------------+
|              value|
+-------------------+
|firstname, lastname|
|       Rene, Bremer|
|       Thom, Bremer|
|       Stef, Bremer|
+-------------------+

In [77]:
%%spark

val par_sqldw_server = "test-edlprod1-dbs.database.windows.net"
val par_sqldw_db_schema_table = "test-synapse-sqldw.dbo.AdultCensusIncome4"

// SQLDW outside Synapse analytics. In this example, SQLDW table data originates from csv in this folder: AdultCensusIncome_withheader.csv. Make sure that the Synapse Workspace MI is registered as external user in the external SQLDWimport com.microsoft.spark.sqlanalytics.utils.Constants
import org.apache.spark.sql.SqlAnalyticsConnector._

val df = spark.read.
option(Constants.SERVER, par_sqldw_server).
sqlanalytics(par_sqldw_db_schema_table)
df.createOrReplaceTempView("temp_table")

StatementMeta(test, 2, 89, Finished, Available)

par_sqldw_server: String = test-edlprod1-dbs.database.windows.net
par_sqldw_db_schema_table: String = test-synapse-sqldw.dbo.AdultCensusIncome4
import org.apache.spark.sql.SqlAnalyticsConnector._
df: org.apache.spark.sql.DataFrame = [age: int, workclass: string ... 13 more fields]


In [78]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# To be able to complete this lab in under an hour, let's just work with 1,500k rows
data_all = spark.sql("SELECT * FROM temp_table").limit(15000000)
#sample.createOrReplaceTempView("AdultCensusIncome")

columns_new = [col.replace("-", "") for col in data_all.columns]
data_all = data_all.toDF(*columns_new)

StatementMeta(test, 2, 90, Finished, Available)

In [79]:
data_all.printSchema()

StatementMeta(test, 2, 91, Finished, Available)

root
 |-- age: integer (nullable = false)
 |-- workclass: string (nullable = false)
 |-- fnlwgt: integer (nullable = false)
 |-- education: string (nullable = false)
 |-- educationnum: integer (nullable = false)
 |-- maritalstatus: string (nullable = false)
 |-- occupation: string (nullable = false)
 |-- relationship: string (nullable = false)
 |-- race: string (nullable = false)
 |-- sex: string (nullable = false)
 |-- capitalgain: integer (nullable = false)
 |-- capitalloss: integer (nullable = false)
 |-- hoursperweek: integer (nullable = false)
 |-- nativecountry: string (nullable = false)
 |-- income: string (nullable = false)

In [80]:
# In case public access is disabled from the storage account attached to Synapse (par_storage), Spark pool can't create the temp table and stack trace below is created
# In case public access is enabled in the storage account attached to Synapse (par_storage), temp table can be created on storage account and data can be loaded correctly
#
# It seems that org.apache.spark.sql.SqlAnalyticsConnector._ tries to acces to storage account using the public DNS rather than the private DNS

data_all.limit(10).show()

StatementMeta(test, 2, 92, Submitted, Running)